# 03 - TFX Interactive Training Pipeline Execution

The purpose of this notebook is to interactively run the following `TFX` pipeline steps:
1. Receive hyperparameters using `hyperparam_gen` custom Python component.
2. Extract data from BigQuery using `BigQueryExampleGen` component.
3. Validate the raw data using `StatisticsGen` and `ExampleValidator` components.
4. Process the data using `Transform` component.
5. Train a custom model using `Trainer` component.
7. Evaluate and Validate the custom model using `ModelEvaluator` component.
7. Save the blessed to model registry location using `Pusher` component.
8. Upload the model to Vertex AI using `vertex_model_pusher` custom Python component

The custom components are implemented in the [tfx_pipeline/components.py](tfx_pipeline/components) module.

## Setup

### Import libraries

In [1]:
import os
import json
import numpy as np
from tfx import v1 as tfx
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils
import logging

from google.cloud import aiplatform

from src.common import features
from src.model_training import data
from src.tfx_pipelines import components

from tfx.proto import example_gen_pb2, transform_pb2, pusher_pb2

from tfx.v1.types.standard_artifacts import Model, ModelBlessing, Schema
from tfx.v1.dsl import Pipeline, Importer, Resolver, Channel
from tfx.v1.dsl.experimental import LatestBlessedModelStrategy, LatestArtifactStrategy

from tfx.v1.components import (
    StatisticsGen,
    SchemaGen,
    ImportSchemaGen,
    ExampleValidator,
    Transform,
    Evaluator,
    Pusher,
)

logging.getLogger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')

print("TFX Version:", tfx.__version__)
print("Tensorflow Version:", tf.__version__)

2024-01-26 16:40:15.460348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libten

Using TensorFlow backend
TFX Version: 1.14.0
Tensorflow Version: 2.13.1


### Setup Google Cloud project

In [2]:
PROJECT = 'stellar-orb-408015' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET =  'stellar-orb-408015-bucket' # Change to your bucket name.
SERVICE_ACCOUNT = "pipelines-sa@stellar-orb-408015.iam.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "stellar-orb-408015":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "pipelines-sa@stellar-orb-408015.iam.gserviceaccount.com":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "stellar-orb-408015-bucket":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
PARENT = f"projects/{PROJECT}/locations/{REGION}"
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)
print("Vertex API Parent URI:", PARENT)

Creating gs://stellar-orb-408015/...
ServiceException: 409 A Cloud Storage bucket named 'stellar-orb-408015' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

Project ID: stellar-orb-408015
Region: us-central1
Bucket name: stellar-orb-408015
Service Account: 13110252891-compute@developer.gserviceaccount.com
Vertex API Parent URI: projects/stellar-orb-408015/locations/us-central1


### Set configurations

In [3]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'

WORKSPACE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}'
RAW_SCHEMA_DIR = 'src/raw_schema'

MLMD_SQLLITE = 'mlmd.sqllite'
ARTIFACT_STORE = os.path.join(WORKSPACE, 'tfx_artifacts_interactive')
MODEL_REGISTRY = os.path.join(WORKSPACE, 'model_registry')
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)

## Create Interactive Context

In [4]:
REMOVE_ARTIFACTS = True

if tf.io.gfile.exists(ARTIFACT_STORE) and REMOVE_ARTIFACTS:
    print("Removing previous artifacts...")
    tf.io.gfile.rmtree(ARTIFACT_STORE)
    
if tf.io.gfile.exists(MLMD_SQLLITE) and REMOVE_ARTIFACTS:
    print("Deleting previous mlmd.sqllite...")
    tf.io.gfile.rmtree(MLMD_SQLLITE)
    
print(f'Pipeline artifacts directory: {PIPELINE_ROOT}')
print(f'Local metadata SQLlit path: {MLMD_SQLLITE}')

Pipeline artifacts directory: gs://stellar-orb-408015/chicago-taxi-tips/tfx_artifacts_interactive/chicago-taxi-tips-classifier-v01-train-pipeline
Local metadata SQLlit path: mlmd.sqllite


In [5]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = MLMD_SQLLITE
connection_config.sqlite.connection_mode = 3 # READWRITE_OPENCREATE
mlmd_store = mlmd.metadata_store.MetadataStore(connection_config)

context = InteractiveContext(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=connection_config
)

## 1. Hyperparameter generation

In [6]:
hyperparams_gen = components.hyperparameters_gen(
    num_epochs=5,
    learning_rate=0.001,
    batch_size=512,
)

context.run(hyperparams_gen, enable_cache=False)

ExecutionResult(
    component_id: hyperparameters_gen
    execution_id: 1
    outputs:
        hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=hyperparameters_gen, output_key=hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [7]:
json.load(
    tf.io.gfile.GFile(
        os.path.join(
            hyperparams_gen.outputs['hyperparameters'].get()[0].uri, 'hyperparameters.json')
    )
)

{'num_epochs': 5, 'batch_size': 512, 'learning_rate': 0.001}

## 2. Data extraction

In [8]:
from src.common import datasource_utils
from tfx.proto import example_gen_pb2, transform_pb2

### Extract train and eval splits

In [9]:
from tfx import v1 as tfx

sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='UNASSIGNED', limit=5000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
        ]
    )
)

train_example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(query=sql_query, output_config=output_config,
                                                                             custom_config=json.dumps({}))

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    train_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

/opt/conda/lib/python3.10/site-packages/apache_beam/io/gcp/bigquery.py:2748: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Extract test split

In [10]:
sql_query = datasource_utils.get_training_source_query(
    PROJECT, REGION, DATASET_DISPLAY_NAME, ml_use='TEST', limit=1000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
        ]
    )
)

test_example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(query=sql_query, output_config=output_config,
                                                                            custom_config=json.dumps({}))

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location={os.path.join(WORKSPACE, 'tmp')}"
]

context.run(
    test_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Read sample extract tfrecords

In [11]:
train_uri = os.path.join(train_example_gen.outputs['examples'].get()[0].uri, "Split-train/*")
source_raw_schema = tfdv.load_schema_text(os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt'))
raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec

def _parse_tf_example(tfrecord):
    return tf.io.parse_single_example(tfrecord, raw_feature_spec)

tfrecord_filenames = tf.data.Dataset.list_files(train_uri)
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
dataset = dataset.map(_parse_tf_example)

for raw_features in dataset.shuffle(1000).batch(3).take(1):
    for key in raw_features:
        print(f"{key}: {np.squeeze(raw_features[key], -1)}")
    print("")

dropoff_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
euclidean: [   0.        0.     1618.4586]
loc_cross: [b'POINT(-87.7 42)POINT(-87.7 42)' b'POINT(-87.7 42)POINT(-87.7 42)'
 b'POINT(-87.7 42)POINT(-87.7 42)']
payment_type: [b'Cash' b'Cash' b'Cash']
pickup_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
tip_bin: [0 0 0]
trip_day: [13 10 10]
trip_day_of_week: [5 2 6]
trip_hour: [20 17 23]
trip_miles: [0.93 0.24 1.4 ]
trip_month: [2 2 1]
trip_seconds: [296 103 420]



## 3. Data validation

### Import raw schema

In [12]:
schema_importer = Importer(
    source_uri='src/tfx_model_training/raw_schema',
    artifact_type=Schema,
    reimport=False
)

context.run(schema_importer)

ExecutionResult(
    component_id: Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "src/tfx_model_training/raw_schema"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.14.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
            _input_trigger: None
        ))

### Generate statistics

In [13]:
statistics_gen = StatisticsGen(
    examples=train_example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
!rm -r {RAW_SCHEMA_DIR}/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


### Validate statistics against schema

In [15]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [16]:
context.show(example_validator.outputs['anomalies'])

## 4. Data transformation

In [17]:
_transform_module_file = 'src/preprocessing/transformations.py'


transform = tfx.components.Transform(
    examples=train_example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module_file,
    splits_config=transform_pb2.SplitsConfig(
        analyze=['train'], transform=['train', 'eval']),
)

context.run(transform, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib
installing to /var/tmp/tmpek7pynqu
running install
running install_lib
copying build/lib/transformations.py -> /var/tmp/tmpek7pynqu
copying build/lib/etl.py -> /var/tmp/tmpek7pynqu
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmpek7pynqu/tfx_user_code_Transform-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187cc6d3ca9-py3.10.egg-

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing /var/tmp/tmp5_hxp045/tfx_user_code_Transform-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187cc6d3ca9-py3-none-any.whl


Processing /var/tmp/tmpvg0716gp/tfx_user_code_Transform-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187cc6d3ca9-py3-none-any.whl


Processing /var/tmp/tmpzboqt86_/tfx_user_code_Transform-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187cc6d3ca9-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Read sample transformed tfrecords

In [18]:
transformed_train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, "Split-train/*")
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

tft_output = tft.TFTransformOutput(transform_graph_uri)
transform_feature_spec = tft_output.transformed_feature_spec()

for input_features, target in data.get_dataset(
    transformed_train_uri, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} ({input_features[key].dtype}): {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

dropoff_grid_xf (<dtype: 'int64'>): [0, 0, 0]
euclidean_xf (<dtype: 'float32'>): [1.6763962507247925, 0.9885498881340027, 0.39919888973236084]
loc_cross_xf (<dtype: 'int64'>): [0, 0, 0]
payment_type_xf (<dtype: 'int64'>): [0, 3, 0]
pickup_grid_xf (<dtype: 'int64'>): [0, 0, 0]
trip_day_of_week_xf (<dtype: 'int64'>): [5, 0, 1]
trip_day_xf (<dtype: 'int64'>): [2, 0, 17]
trip_hour_xf (<dtype: 'int64'>): [17, 7, 5]
trip_miles_xf (<dtype: 'float32'>): [1.2284228801727295, 2.2774879932403564, 0.2498832494020462]
trip_month_xf (<dtype: 'int64'>): [4, 5, 7]
trip_seconds_xf (<dtype: 'float32'>): [0.16689293086528778, 0.24458394944667816, 0.005697291810065508]
target: [0, 0, 0]


In [19]:
schema = tft_output.transformed_metadata.schema
schema

feature {
  name: "dropoff_grid_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "euclidean_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "loc_cross_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "payment_type_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "pickup_grid_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "tip_bin"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "trip_day_of_week_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "trip_day_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "trip_hour_xf"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
  }
}
feature {
  name: "trip_miles_xf"
  type: FLOAT
  presence {
    min_fraction: 1.0
  }
  

## 5. Model training

### Get the latest model to warm start

In [20]:
latest_model_resolver = Resolver(
      strategy_class=LatestArtifactStrategy,
      model=Channel(type=Model),
  ).with_id('latest_model_resolver')
context.run(latest_model_resolver, enable_cache=False)

ExecutionResult(
    component_id: latest_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [21]:
#latest_model_resolver = Resolver(
#    strategy_class=latest_artifacts_resolver.LatestArtifactsResolver,
#    latest_model=Channel(type=tfx.types.standard_artifacts.Model)
#)

#context.run(latest_model_resolver, enable_cache=False)

### Train the model

In [22]:
_train_module_file = 'src/tfx_model_training/model_runner.py'

trainer = tfx.components.Trainer(
    module_file=_train_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    #base_model=latest_model_resolver.outputs['latest_model'],
    #base_model=latest_model_resolver.outputs['model'],
    transform_graph=transform.outputs['transform_graph'],
    hyperparameters=hyperparams_gen.outputs['hyperparameters'],
    train_args=tfx.proto.TrainArgs(num_steps=10),
    eval_args=tfx.proto.EvalArgs(num_steps=5),
)

context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying defaults.py -> build/lib
copying features.py -> build/lib
copying model_input.py -> build/lib
copying model.py -> build/lib
copying model_exporter.py -> build/lib
copying model_runner.py -> build/lib
copying model_trainer.py -> build/lib
installing to /var/tmp/tmpjyww5pwz
running install


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running install_lib
copying build/lib/model_input.py -> /var/tmp/tmpjyww5pwz
copying build/lib/model.py -> /var/tmp/tmpjyww5pwz
copying build/lib/model_trainer.py -> /var/tmp/tmpjyww5pwz
copying build/lib/defaults.py -> /var/tmp/tmpjyww5pwz
copying build/lib/model_runner.py -> /var/tmp/tmpjyww5pwz
copying build/lib/features.py -> /var/tmp/tmpjyww5pwz
copying build/lib/model_exporter.py -> /var/tmp/tmpjyww5pwz
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmpjyww5pwz/tfx_user_code_Trainer-0.0+36304a6a7fd1322

Processing /var/tmp/tmp5rkolu6c/tfx_user_code_Trainer-0.0+36304a6a7fd13229219c55e192f859b875736ca0869f2d0fba40628b83103309-py3-none-any.whl


5/5 [==============================] - 0s 40ms/step - loss: 0.5261 - accuracy: 0.9117


ExecutionResult(
    component_id: Trainer
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
schema_gen = ImportSchemaGen(schema_file='src/tfx_model_training/raw_schema/schema.pbtxt')
context.run(schema_gen, enable_cache=False)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 10
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [24]:
#_train_module_file = 'src/model_training/runner.py'
_train_module_file = 'src/tfx_model_training/model_runner.py'

# NEW: Configuration for Vertex AI Training.
  # This dictionary will be passed as `CustomJobSpec`.
vertex_job_spec = {
      'project': PROJECT,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }


trainer = tfx.extensions.google_cloud_ai_platform.Trainer(
    module_file=_train_module_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    #base_model=latest_model_resolver.outputs['latest_model'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.TrainArgs(num_steps=10),
    eval_args=tfx.proto.EvalArgs(num_steps=5),
    hyperparameters=hyperparams_gen.outputs['hyperparameters'],
    custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              REGION,
          tfx.extensions.google_cloud_ai_platform.TRAINING_ARGS_KEY:
              vertex_job_spec,

      })


context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying defaults.py -> build/lib
copying features.py -> build/lib
copying model_input.py -> build/lib
copying model.py -> build/lib
copying model_exporter.py -> build/lib
copying model_runner.py -> build/lib
copying model_trainer.py -> build/lib
installing to /var/tmp/tmpkwy09_em
running install


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running install_lib
copying build/lib/model_input.py -> /var/tmp/tmpkwy09_em
copying build/lib/model.py -> /var/tmp/tmpkwy09_em
copying build/lib/model_trainer.py -> /var/tmp/tmpkwy09_em
copying build/lib/defaults.py -> /var/tmp/tmpkwy09_em
copying build/lib/model_runner.py -> /var/tmp/tmpkwy09_em
copying build/lib/features.py -> /var/tmp/tmpkwy09_em
copying build/lib/model_exporter.py -> /var/tmp/tmpkwy09_em
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmpkwy09_em/tfx_user_code_Trainer-0.0+36304a6a7fd1322

ExecutionResult(
    component_id: Trainer
    execution_id: 11
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 6. Model evaluation

### Get the latest blessed model for model validation.

In [25]:
blessed_model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
)

context.run(blessed_model_resolver, enable_cache=False)

ExecutionResult(
    component_id: Resolver
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [26]:
blessed_model_resolver.outputs['model']

.type_name,Model
._artifacts,[]


### Evaluate and validate the model against the baseline model.

In [27]:
#from tfx.components import Evaluator

In [28]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_tf_example',
            label_key=features.TARGET_FEATURE_NAME,
            prediction_key='probabilities')
    ],
    slicing_specs=[
        tfma.SlicingSpec(),
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[   
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.8}),
                        # Change threshold will be ignored if there is no
                        # baseline model resolved from MLMD (first run).
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10}))),
        ])
    ])


evaluator = Evaluator(
    examples=test_example_gen.outputs['examples'],
    example_splits=['test'],
    model=trainer.outputs['model'],
    #baseline_model=blessed_model_resolver.outputs['model'],
    baseline_model=blessed_model_resolver.outputs['model'],
    eval_config=eval_config,
    schema=schema_gen.outputs['schema']
)

context.run(evaluator, enable_cache=False)

ExecutionResult(
    component_id: Evaluator
    execution_id: 13
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [29]:
evaluation_results = evaluator.outputs['evaluation'].get()[0].uri
print("validation_ok:", tfma.load_validation_result(evaluation_results).validation_ok, '\n')

for entry in list(tfma.load_metrics(evaluation_results))[0].metric_keys_and_values:
    value = entry.value.double_value.value
    if value:
        print(entry.key.name, ":", round(entry.value.double_value.value, 3))

validation_ok: True 

accuracy : 0.91
loss : 0.881
example_count : 1000.0
binary_accuracy : 0.91


## 7.1.1 Model pushing

In [30]:
exported_model_location = os.path.join(MODEL_REGISTRY, MODEL_DISPLAY_NAME)

push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=exported_model_location,
    )
)

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=push_destination
)

context.run(pusher, enable_cache=False)

ExecutionResult(
    component_id: Pusher
    execution_id: 14
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 7.1.2. Model Upload to Vertex AI

In [32]:
serving_runtime = 'tf2-cpu.2-12'
serving_image_uri = f"us-docker.pkg.dev/vertex-ai/prediction/{serving_runtime}:latest"

labels = {
    'dataset_name': DATASET_DISPLAY_NAME,
    'pipeline_name': PIPELINE_NAME
}
labels = json.dumps(labels)


vertex_model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=EXPORT_DIR,
    serving_container_image_uri=SERVING_IMAGE,
    parameters_schema_uri=None,
    instance_schema_uri=None,
    explanation_metadata=explanation_metadata,
    explanation_parameters=explanation_parameters,
    labels={
        'dataset_name': DATASET_DISPLAY_NAME,
        'experiment': run_id
    }
)

vertex_model_uploader = components.vertex_model_uploader(
    project=PROJECT,
    region=REGION,
    model_display_name=MODEL_DISPLAY_NAME,
    pushed_model_location=exported_model_location,
    serving_image_uri=serving_image_uri,
    model_blessing=evaluator.outputs['blessing'],
    explanation_config='',
    labels=labels
)

context.run(vertex_model_uploader, enable_cache=False)

NameError: name 'EXPORT_DIR' is not defined

In [ ]:
vertex_model_uploader.outputs['uploaded_model'].get()[0].get_string_custom_property('model_uri')

## 7.2. Model Upload to Vertex AI & Create and Endpoint for online prediction & explanation

create a Vertex AI Model and a Vertex AI Endpoint using the
trained model directly using tfx.extensions.google_cloud_ai_platform.Pusher

In [ ]:

FEATURE_NAMES = [
    "trip_month",
    "trip_day",
    "trip_day_of_week",
    "trip_hour",
    "trip_seconds",
    "trip_miles",
    "payment_type",
    "pickup_grid",
    "dropoff_grid",
    "euclidean",
    "loc_cross",
]

TARGET_FEATURE_NAME = "tip_bin"

TARGET_LABELS = ["tip<20%", "tip>=20%"]

NUMERICAL_FEATURE_NAMES = [
    "trip_seconds",
    "trip_miles",
    "euclidean",
]

EMBEDDING_CATEGORICAL_FEATURES = {
    "trip_month": 2,
    "trip_day": 4,
    "trip_hour": 3,
    "pickup_grid": 3,
    "dropoff_grid": 3,
    "loc_cross": 10,
}

ONEHOT_CATEGORICAL_FEATURE_NAMES = ["payment_type", "trip_day_of_week"]



In [ ]:
def generate_explanation_config():
    explanation_config = {
        "inputs": {},
        "outputs": {},
    }

    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERICAL_FEATURE_NAMES:
            explanation_config["inputs"][feature_name] = {
                "input_tensor_name": feature_name
            }
        else:
            explanation_config["inputs"][feature_name] = {
                "input_tensor_name": feature_name,
            }

    explanation_config["outputs"] = {"scores": {"output_tensor_name": "scores"}}

    return explanation_config


In [33]:
from src.tfx_model_training import features
explanation_metadata = features.generate_explanation_config()
explanation_metadata

{'inputs': {'trip_month': {'input_tensor_name': 'trip_month'},
  'trip_day': {'input_tensor_name': 'trip_day'},
  'trip_day_of_week': {'input_tensor_name': 'trip_day_of_week'},
  'trip_hour': {'input_tensor_name': 'trip_hour'},
  'trip_seconds': {'input_tensor_name': 'trip_seconds'},
  'trip_miles': {'input_tensor_name': 'trip_miles'},
  'payment_type': {'input_tensor_name': 'payment_type'},
  'pickup_grid': {'input_tensor_name': 'pickup_grid'},
  'dropoff_grid': {'input_tensor_name': 'dropoff_grid'},
  'euclidean': {'input_tensor_name': 'euclidean'},
  'loc_cross': {'input_tensor_name': 'loc_cross'}},
 'outputs': {'scores': {'output_tensor_name': 'scores'}}}

In [34]:
# Configuration for pusher.
# Name of Vertex AI Endpoint.
ENDPOINT_NAME = 'prediction-explain' + PIPELINE_NAME

vertex_serving_spec = {
      'project_id': PROJECT,
      'endpoint_name': ENDPOINT_NAME,
      # Remaining argument is passed to aiplatform.Model.deploy()
      # See https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#deploy_the_model
      # for the detail.
      #
      # Machine type is the compute resource to serve prediction requests.
      # See https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types
      # for available machine types and acccerators.
      'machine_type': 'n1-standard-4',
      #'explanation_metadata' : explanation_metadata,
      #'explanation_parameters' : explanation_parameters
      'explanation_metadata': explanation_metadata,
      'explanation_parameters' : {
              "sampled_shapley_attribution": {
                  "path_count": 10
              }  
      }
}



  # Vertex AI provides pre-built containers with various configurations for
  # serving.
  # See https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
  # for available container images.
serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest'


# NEW: Pushes the model to Vertex AI.
pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      model_blessing = evaluator.outputs['blessing'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              REGION,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

context.run(pusher, enable_cache=False)

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/13110252891/locations/us-central1/models/3107375990746120192/operations/2369179208300625920


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/13110252891/locations/us-central1/models/3107375990746120192/operations/2369179208300625920


Model created. Resource name: projects/13110252891/locations/us-central1/models/3107375990746120192@1


...

Deploy Endpoint model backing LRO: projects/13110252891/locations/us-central1/endpoints/8119121513963061248/operations/9005233259231051776


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/13110252891/locations/us-central1/endpoints/8119121513963061248/operations/9005233259231051776


Endpoint model deployed. Resource name: projects/13110252891/locations/us-central1/endpoints/8119121513963061248


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/13110252891/locations/us-central1/endpoints/8119121513963061248


ExecutionResult(
    component_id: Pusher
    execution_id: 15
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Test with a prediction request

Once the pipeline completes, you will find a *deployed* model at the one of the
endpoints in 'Vertex AI > Endpoints'. We need to know the id of the endpoint to
send a prediction request to the new endpoint. This is different from the
*endpoint name* we entered above. You can find the id at the [Endpoints page](https://console.cloud.google.com/vertex-ai/endpoints) in
`Google Cloud Console`, it looks like a very long number.

**Set ENDPOINT_ID below before running it.**


In [35]:
ENDPOINT_ID='8119121513963061248'     # Replace this with your ENDPOINT_ID
if not ENDPOINT_ID:
    from absl import logging
    logging.error('Please set the endpoint id.')

In [ ]:
aiplatform.Endpoint.list(
    filter=f'display_name={ENDPOINT_NAME}', 
    order_by="update_time")[-1].gca_resource.deployed_models[0].id

In [ ]:
aiplatform.Endpoint.list(
    filter=f'display_name={ENDPOINT_NAME}', 
    order_by="update_time")[-1].gca_resource

In [ ]:

endpoint_name = aiplatform.Endpoint.list(
    filter=f'display_name={ENDPOINT_NAME}', 
    order_by="update_time")[-1].gca_resource.name

In [ ]:
endpoint_name

In [36]:
# TODO 4
# docs_infra: no_execute
import numpy as np
from google.cloud import aiplatform
# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.

instances = [  
    {
        "dropoff_grid": ["POINT(-87.6 42)"],
        "euclidean": [0.0],
        "loc_cross": ["POINT(-87.6 42)POINT(-87.6 42)"],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 42)"],
        "trip_miles": [1.02],
        "trip_day": [9],
        "trip_hour": [16],
        "trip_month": [1],
        "trip_day_of_week": [5],
        "trip_seconds": [582]
    }
]

endpoint = client.endpoint_path(
    project=PROJECT,
    location=REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

# Uses argmax to find the index of the maximum value.
print('classes:', response.predictions[0]['classes'][np.argmax(response.predictions[0]['scores'])])

classes: tip>=20%


In [37]:
response

predictions {
  struct_value {
    fields {
      key: "classes"
      value {
        list_value {
          values {
            string_value: "tip<20%"
          }
          values {
            string_value: "tip>=20%"
          }
        }
      }
    }
    fields {
      key: "scores"
      value {
        list_value {
          values {
            number_value: 0.451987118
          }
          values {
            number_value: 0.548012853
          }
        }
      }
    }
  }
}
deployed_model_id: "7946836837983584256"
model: "projects/13110252891/locations/us-central1/models/3107375990746120192"
model_display_name: "v1706265899"
model_version_id: "1"

In [38]:
instances_2 = [  
    {
        "dropoff_grid": ["POINT(-87.7 42)"],
        "euclidean": [1323.1249247549483],
        "loc_cross": ["POINT(-87.6 42)POINT(-87.7 42)"],
        "payment_type": ["Cash"],
        "pickup_grid": ["POINT(-87.6 42)"],
        "trip_miles": [0.6],
        "trip_day": [12],
        "trip_hour": [10],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [360]
    }
]

In [39]:
# Send a prediction request and get response.
response_2 = client.predict(endpoint=endpoint, instances=instances_2)

# Uses argmax to find the index of the maximum value.
print('classes:', response_2.predictions[0]['classes'][np.argmax(response_2.predictions[0]['scores'])])

classes: tip>=20%


In [40]:
response_2

predictions {
  struct_value {
    fields {
      key: "classes"
      value {
        list_value {
          values {
            string_value: "tip<20%"
          }
          values {
            string_value: "tip>=20%"
          }
        }
      }
    }
    fields {
      key: "scores"
      value {
        list_value {
          values {
            number_value: 0.447076499
          }
          values {
            number_value: 0.552923501
          }
        }
      }
    }
  }
}
deployed_model_id: "7946836837983584256"
model: "projects/13110252891/locations/us-central1/models/3107375990746120192"
model_display_name: "v1706265899"
model_version_id: "1"

In [41]:
explain_response = client.explain(endpoint=endpoint, instances=instances_2)

In [42]:
explain_response

explanations {
  attributions {
    baseline_output_value: 0.6117322444915771
    instance_output_value: 0.5529235005378723
    feature_attributions {
      struct_value {
        fields {
          key: "dropoff_grid"
          value {
            list_value {
              values {
                number_value: -0.0002007484436035156
              }
            }
          }
        }
        fields {
          key: "euclidean"
          value {
            list_value {
              values {
                number_value: -0.001150798797607422
              }
            }
          }
        }
        fields {
          key: "loc_cross"
          value {
            list_value {
              values {
                number_value: 8.323192596435547e-05
              }
            }
          }
        }
        fields {
          key: "payment_type"
          value {
            list_value {
              values {
                number_value: -0.002041327953338623
              }
 

### Create and Run the pipeline on Vertex Pipelines

In [18]:
from src.tfx_pipelines import config
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = 'modifiedpipeline'
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/test"
os.environ["TRAIN_LIMIT"] = "10"#"85000"
os.environ["TEST_LIMIT"] = "12"#"15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["TFX_IMAGE_URI"] = f"gcr.io/{PROJECT}/{DATASET_DISPLAY_NAME}:02"
os.environ["ENABLE_CACHE"] = "1"

PIPELINE_DEFINITION_FILE = f'{config.PIPELINE_NAME}' + 'info_pipeline.json'


In [19]:
os.environ["REGION"]

'us-central1'

In [20]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

PROJECT: stellar-orb-408015
REGION: us-central1
GCS_LOCATION: gs://stellar-orb-408015/chicago-taxi-tips/test
ARTIFACT_STORE_URI: gs://stellar-orb-408015/chicago-taxi-tips/test/tfx_artifacts
MODEL_REGISTRY_URI: model_registry
DATASET_DISPLAY_NAME: chicago-taxi-tips
MODEL_DISPLAY_NAME: chicago-taxi-tips-classifier-v01
PIPELINE_NAME: modifiedpipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 10
TEST_LIMIT: 12
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: 0.8
USE_KFP_SA: False
TFX_IMAGE_URI: gcr.io/stellar-orb-408015/chicago-taxi-tips:02
BEAM_RUNNER: DataflowRunner
BEAM_DIRECT_PIPELINE_ARGS: ['--project=stellar-orb-408015', '--temp_location=gs://stellar-orb-408015/chicago-taxi-tips/test/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=stellar-orb-408015', '--temp_location=gs://stellar-orb-408015/chicago-taxi-tips/test/temp', '--region=us-central1', '--runner=DataflowRunner']
TRAINING_RUNNER: vertex
VERTEX_TRAINING_ARGS: {'project': 

In [21]:
config.REGION

'us-central1'

In [22]:
from tfx.v1.extensions.google_cloud_ai_platform import Trainer as VertexTrainer 
from tfx.v1.extensions.google_cloud_ai_platform import Pusher as VertexPrediction 
from tfx.orchestration import pipeline, data_types
from src.tfx_pipelines import config
from src.common import features, datasource_utils

RAW_SCHEMA_DIR = "src/tfx_model_training/raw_schema"
TRANSFORM_MODULE_FILE = "src/preprocessing/transformations.py"
TRAIN_MODULE_FILE = "src/tfx_model_training/model_runner.py"

def _create_pipeline(
    pipeline_root: str,
    num_epochs: data_types.RuntimeParameter,
    batch_size: data_types.RuntimeParameter,
    learning_rate: data_types.RuntimeParameter,
    #metadata_connection_config: metadata_store_pb2.ConnectionConfig = None,
)-> tfx.dsl.Pipeline:

    # Hyperparameter generation.
    hyperparams_gen = components.hyperparameters_gen(
        num_epochs=num_epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
    ).with_id("HyperparamsGen")

    # Get train source query.
    train_sql_query = datasource_utils.get_training_source_query(
        project=config.PROJECT,
        region=config.REGION,
        dataset_display_name=config.DATASET_DISPLAY_NAME,
        ml_use="UNASSIGNED",
        limit=int(config.TRAIN_LIMIT),
    )

    train_output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(
            splits=[
                example_gen_pb2.SplitConfig.Split(
                    name="train", hash_buckets=int(config.NUM_TRAIN_SPLITS)
                ),
                example_gen_pb2.SplitConfig.Split(
                    name="eval", hash_buckets=int(config.NUM_EVAL_SPLITS)
                ),
            ]
        )
    )

    # Train example generation
    
    train_example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(
        query=train_sql_query,
        output_config=train_output_config,
        custom_config=json.dumps({})
    ).with_id("TrainDataGen")

    # Get test source query.
    test_sql_query = datasource_utils.get_training_source_query(
        config.PROJECT,
        config.REGION,
        config.DATASET_DISPLAY_NAME,
        ml_use="TEST",
        limit=int(config.TEST_LIMIT),
    )

    test_output_config = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(
            splits=[
                example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
            ]
        )
    )

    # Test example generation.
    test_example_gen = tfx.extensions.google_cloud_big_query.BigQueryExampleGen(
        query=test_sql_query,
        output_config=test_output_config,
        custom_config=json.dumps({})
    ).with_id("TestDataGen")

    # Schema importer.
    schema_importer = Importer(
        source_uri=RAW_SCHEMA_DIR,
        artifact_type=Schema,
    ).with_id("SchemaImporter")


    # Statistics generation.
    statistics_gen = StatisticsGen(examples=train_example_gen.outputs["examples"]).with_id(
        "StatisticsGen"
    )

    # Example validation.
    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_importer.outputs["result"],
    ).with_id("ExampleValidator")

    # Data transformation.
    transform = Transform(
        examples=train_example_gen.outputs["examples"],
        schema=schema_importer.outputs["result"],
        module_file=TRANSFORM_MODULE_FILE,
        # This is a temporary workaround to run on Dataflow.
        force_tf_compat_v1=config.BEAM_RUNNER == "DataflowRunner",
        splits_config=transform_pb2.SplitsConfig(
            analyze=["train"], transform=["train", "eval"]
        ),
    ).with_id("DataTransformer")

    # Add dependency from example_validator to transform.
    transform.add_upstream_node(example_validator)

    # Get the latest model to warmstart
    warmstart_model_resolver = Resolver(
        strategy_class=LatestArtifactStrategy,
        model=Channel(type=Model),
        ).with_id("WarmstartModelResolver")
    
    # Add the ImportSchemaGen so that it will add it to the GCS bucket
    schema_gen = ImportSchemaGen(schema_file='src/tfx_model_training/raw_schema/schema.pbtxt')
    
    # Model training.
    if config.TRAINING_RUNNER == "vertex":
        trainer = VertexTrainer(
            module_file=TRAIN_MODULE_FILE,
            examples=transform.outputs["transformed_examples"],
            schema=schema_gen.outputs['schema'],
            base_model=warmstart_model_resolver.outputs["model"],
            transform_graph=transform.outputs["transform_graph"],
            train_args=tfx.proto.TrainArgs(num_steps=10),
            eval_args=tfx.proto.EvalArgs(num_steps=5),
            hyperparameters=hyperparams_gen.outputs["hyperparameters"],
            custom_config=config.VERTEX_TRAINING_CONFIG
        ).with_id("ModelTrainer")
    else :
        trainer = Trainer(
            module_file=TRAIN_MODULE_FILE,
            examples=transform.outputs["transformed_examples"],
            schema=schema_importer.outputs["result"],
            base_model=warmstart_model_resolver.outputs["model"],
            train_args=tfx.proto.TrainArgs(num_steps=10),
            eval_args=tfx.proto.EvalArgs(num_steps=5),
            transform_graph=transform.outputs["transform_graph"],
            hyperparameters=hyperparams_gen.outputs["hyperparameters"],
        ).with_id("ModelTrainer")
        

    # Get the latest blessed model (baseline) for model validation.
    baseline_model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing)
    ).with_id("BaselineModelResolver")

    # Prepare evaluation config.
    eval_config = tfma.EvalConfig(
        model_specs=[
            tfma.ModelSpec(
                signature_name="serving_tf_example",
                label_key=features.TARGET_FEATURE_NAME,
                prediction_key="probabilities",
            )
        ],
        slicing_specs=[
            tfma.SlicingSpec(),
        ],
        metrics_specs=[
            tfma.MetricsSpec(
                metrics=[
                    tfma.MetricConfig(class_name="ExampleCount"),
                    tfma.MetricConfig(
                        class_name="BinaryAccuracy",
                        threshold=tfma.MetricThreshold(
                            value_threshold=tfma.GenericValueThreshold(
                                lower_bound={"value": float(config.ACCURACY_THRESHOLD)}
                            ),
                            # Change threshold will be ignored if there is no
                            # baseline model resolved from MLMD (first run).
                            change_threshold=tfma.GenericChangeThreshold(
                                direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                absolute={"value": -1e-10},
                            ),
                        ),
                    ),
                ]
            )
        ],
    )

    # Model evaluation.
    evaluator = Evaluator(
        examples=test_example_gen.outputs["examples"],
        example_splits=["test"],
        model=trainer.outputs["model"],
        baseline_model=baseline_model_resolver.outputs["model"],
        eval_config=eval_config,
        schema=schema_gen.outputs['schema'],
    ).with_id("ModelEvaluator")


    exported_model_location = os.path.join(
        config.MODEL_REGISTRY_URI, config.MODEL_DISPLAY_NAME
    )
    push_destination = pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=exported_model_location
        )
    )

    # Push custom model to model registry.
    #pusher = Pusher(
    #    model=trainer.outputs["model"],
    #    model_blessing=evaluator.outputs["blessing"],
    #    push_destination=push_destination,
    #).with_id("ModelPusher")
    
    # Upload custom trained model to Vertex AI.
    labels = {
        "dataset_name": config.DATASET_DISPLAY_NAME,
        "pipeline_name": config.PIPELINE_NAME,
        "pipeline_root": pipeline_root
    }
    labels = json.dumps(labels)
    
    #explanation_config = json.dumps(features.generate_explanation_config())
    
    #vertex_model_uploader = custom_components.vertex_model_uploader(
    #   project=config.PROJECT,
    #    region=config.REGION,
    #    model_display_name=config.MODEL_DISPLAY_NAME,
    #    pushed_model_location=exported_model_location,
    #    serving_image_uri=config.SERVING_IMAGE_URI,
    #    model_blessing=evaluator.outputs["blessing"],
    #    explanation_config=explanation_config,
    #    labels=labels
    #).with_id("VertexUploader")

    pusher = VertexPrediction(
        model=trainer.outputs['model'],
        model_blessing = evaluator.outputs['blessing'],
        custom_config=config.VERTEX_PREDICTION_CONFIG)
   
    pipeline_components = [
        hyperparams_gen,
        train_example_gen,
        test_example_gen,
        statistics_gen,
        schema_importer,
        example_validator,
        transform,
        warmstart_model_resolver,
        schema_gen,
        trainer,
        baseline_model_resolver,
        evaluator,
        pusher,
    ]

    #if int(config.UPLOAD_MODEL):
    #    pipeline_components.append(vertex_model_uploader)
    #    # Add dependency from pusher to aip_model_uploader.
    #    vertex_model_uploader.add_upstream_node(pusher)

    logging.info(
        f"Pipeline components: {[component.id for component in pipeline_components]}"
    )

    beam_pipeline_args = config.BEAM_DIRECT_PIPELINE_ARGS
    if config.BEAM_RUNNER == "DataflowRunner":
        beam_pipeline_args = config.BEAM_DATAFLOW_PIPELINE_ARGS

    logging.info(f"Beam pipeline args: {beam_pipeline_args}")

    return tfx.dsl.Pipeline(
        pipeline_name=config.PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=pipeline_components,
        beam_pipeline_args=beam_pipeline_args,
        #metadata_connection_config=metadata_connection_config,
        #enable_cache=int(config.ENABLE_CACHE),
    )


In [23]:
import os
from kfp.v2.google.client import AIPlatformClient
from tfx.orchestration import data_types
from tfx.orchestration.kubeflow.v2 import kubeflow_v2_dag_runner
from src.tfx_model_training import defaults
pipeline_definition_file = PIPELINE_DEFINITION_FILE
pipeline_root = os.path.join(
        config.ARTIFACT_STORE_URI,
        config.PIPELINE_NAME,
    )

managed_pipeline = _create_pipeline(
        pipeline_root=pipeline_root,
        num_epochs=data_types.RuntimeParameter(
            name="num_epochs",
            default=defaults.NUM_EPOCHS,
            ptype=int,
        ),
        batch_size=data_types.RuntimeParameter(
            name="batch_size",
            default=defaults.BATCH_SIZE,
            ptype=int,
        ),
        learning_rate=data_types.RuntimeParameter(
            name="learning_rate",
            default=defaults.LEARNING_RATE,
            ptype=float,
        ),

    )

runner = kubeflow_v2_dag_runner.KubeflowV2DagRunner(
        config=kubeflow_v2_dag_runner.KubeflowV2DagRunnerConfig(
            default_image=config.TFX_IMAGE_URI
        ),
        output_filename=pipeline_definition_file,
    )




In [13]:
runner.run(managed_pipeline, write_out=True)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib
installing to /var/tmp/tmprmfw2c7w
running install


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running install_lib
copying build/lib/transformations.py -> /var/tmp/tmprmfw2c7w
copying build/lib/etl.py -> /var/tmp/tmprmfw2c7w
running install_egg_info
running egg_info
creating tfx_user_code_DataTransformer.egg-info
writing tfx_user_code_DataTransformer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_DataTransformer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_DataTransformer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
Copying tfx_user_code_DataTransformer.egg-info to /var/tmp/tmprmfw2c7w/tfx_user_code_DataTransformer-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187cc6d3ca9-py3.10.egg-info
running install_scripts
creating /var/tmp/tmprmfw2c7w/tfx_user_code_DataTransformer-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187c

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running install_lib
copying build/lib/model_input.py -> /var/tmp/tmpf44g4mkk
copying build/lib/model.py -> /var/tmp/tmpf44g4mkk
copying build/lib/model_trainer.py -> /var/tmp/tmpf44g4mkk
copying build/lib/defaults.py -> /var/tmp/tmpf44g4mkk
copying build/lib/model_runner.py -> /var/tmp/tmpf44g4mkk
copying build/lib/features.py -> /var/tmp/tmpf44g4mkk
copying build/lib/model_exporter.py -> /var/tmp/tmpf44g4mkk
running install_egg_info
running egg_info
creating tfx_user_code_ModelTrainer.egg-info
writing tfx_user_code_ModelTrainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_ModelTrainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_ModelTrainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_ModelTrainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_ModelTrainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_ModelTrainer.egg-info/SOURCES.txt'
Copying tfx_user_code_ModelTrainer.egg-info to /var/tmp/tmpf44g4mkk/t

{'displayName': 'modifiedpipeline',
 'pipelineSpec': {'root': {'dag': {'tasks': {'Pusher': {'componentRef': {'name': 'Pusher'},
      'dependentTasks': ['ModelEvaluator', 'ModelTrainer'],
      'taskInfo': {'name': 'Pusher'},
      'inputs': {'artifacts': {'model': {'taskOutputArtifact': {'producerTask': 'ModelTrainer',
          'outputArtifactKey': 'model'}},
        'model_blessing': {'taskOutputArtifact': {'outputArtifactKey': 'blessing',
          'producerTask': 'ModelEvaluator'}}},
       'parameters': {'custom_config': {'runtimeValue': {'constantValue': {'stringValue': '{"ai_platform_enable_vertex": true, "ai_platform_serving_args": {"project": "stellar-orb-408015", "worker_pool_specs": [{"container_spec": {"image_uri": "gcr.io/tfx-oss-public/tfx:1.14.0"}, "machine_spec": {"machine_type": "n1-standard-4"}, "replica_count": 1}]}, "ai_platform_vertex_container_image_uri": "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest", "ai_platform_vertex_region": "us-central1"}'}}}

In [24]:
!echo $TFX_IMAGE_URI

gcr.io/stellar-orb-408015/chicago-taxi-tips:02


In [31]:
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8 --timeout=DURATION=1h

Creating temporary tarball archive of 72 file(s) totalling 1.5 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2024.01.26/17.55.02.255294.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://stellar-orb-408015_cloudbuild/source/1706291702.38803-32a70a99bf9546deae795c8bab9d73eb.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/stellar-orb-408015/locations/global/builds/ac912bad-46cb-4158-a5b8-e6c9576c48f7].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/ac912bad-46cb-4158-a5b8-e6c9576c48f7?project=13110252891 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ac912bad-46cb-4158-a5b8-e6c9576c48f7"

FETCHSOURCE
Fetching storage object: gs://stellar-orb-408015_cloudbuild/source/1706291702.38803-32a70a99bf9546de

In [30]:
!pip install pyfarmhash==0.3.2

In [28]:
pipeline_client = AIPlatformClient(project_id=config.PROJECT, region=config.REGION)
pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'num_epochs': 30,
    }
)

{'name': 'projects/13110252891/locations/us-central1/pipelineJobs/modifiedpipeline-20240126153830',
 'displayName': 'modifiedpipeline-20240126153830',
 'createTime': '2024-01-26T15:38:31.120608Z',
 'updateTime': '2024-01-26T15:38:31.120608Z',
 'pipelineSpec': {'deploymentConfig': {'@type': 'type.googleapis.com/ml_pipelines.PipelineDeploymentConfig',
   'executors': {'BaselineModelResolver-model-blessing-resolver_executor': {'resolver': {'outputArtifactQueries': {'model_blessing': {'filter': 'schema_title="tfx.ModelBlessing" AND state=LIVE AND metadata.blessed.number_value=1'}}}},
    'BaselineModelResolver-model-resolver_executor': {'resolver': {'outputArtifactQueries': {'model': {'filter': 'schema_title="tfx.Model" AND state=LIVE AND name="{{$.inputs.artifacts[\'input\'].metadata[\'current_model_id\']}}"'}}}},
    'DataTransformer_executor': {'container': {'image': 'gcr.io/stellar-orb-408015/chicago-taxi-tips:v01',
      'command': ['python',
       '-m',
       'tfx.orchestration.kub

In [27]:
pip freeze 

absl-py==1.4.0
aiofiles==22.1.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1701099469104/work
aiohttp-cors==0.7.0
aiorwlock==1.3.0
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
aiosqlite==0.19.0
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1700835416766/work
apache-beam==2.50.0
archspec @ file:///home/conda/feedstock_root/build_artifacts/archspec_1699370045702/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1695386546427/work
array-record==0.5.0
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
astunparse==1.6.3
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
async-timeout @ file:///home/con

In [10]:
config.PIPELINE_NAME

'modifiedpipeline'

In [11]:
import os

from src.tfx_model_training import defaults


# Path to various pipeline artifact.
#PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(BUCKET, f'{config.PIPELINE_NAME}')
PIPELINE_ROOT = os.path.join(
        config.ARTIFACT_STORE_URI,
        config.PIPELINE_NAME,
    )
runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)

_ = runner.run(
    _create_pipeline(
       pipeline_root=PIPELINE_ROOT,
        num_epochs=data_types.RuntimeParameter(
            name="num_epochs",
            default=defaults.NUM_EPOCHS,
            ptype=int,
        ),
        batch_size=data_types.RuntimeParameter(
            name="batch_size",
            default=defaults.BATCH_SIZE,
            ptype=int,
        ),
        learning_rate=data_types.RuntimeParameter(
            name="learning_rate",
            default=defaults.LEARNING_RATE,
            ptype=float,
        ),

    ))

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to /var/tmp/tmpo8q6kjdt
running install
running install_lib
copying build/lib/transformations.py -> /var/tmp/tmpo8q6kjdt
copying build/lib/etl.py -> /var/tmp/tmpo8q6kjdt
running install_egg_info
running egg_info
creating tfx_user_code_DataTransformer.egg-info
writing tfx_user_code_DataTransformer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_DataTransformer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_DataTransformer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
Copying tfx_user_code_DataTransformer.egg-info to /var/tmp/tmpo8q6kjdt/tfx_user_code_DataTransformer-0.0+51b296fdc6db1615d7014be718135cc309d251adf9aca4e50e35c187cc6d3ca9-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpo8q6kjdt/tfx_user_code_DataTransformer-0.0+51b296

/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


installing to /var/tmp/tmptqz_uhoy
running install
running install_lib
copying build/lib/model_input.py -> /var/tmp/tmptqz_uhoy
copying build/lib/model.py -> /var/tmp/tmptqz_uhoy
copying build/lib/model_trainer.py -> /var/tmp/tmptqz_uhoy
copying build/lib/defaults.py -> /var/tmp/tmptqz_uhoy
copying build/lib/model_runner.py -> /var/tmp/tmptqz_uhoy
copying build/lib/features.py -> /var/tmp/tmptqz_uhoy
copying build/lib/model_exporter.py -> /var/tmp/tmptqz_uhoy
running install_egg_info
running egg_info
creating tfx_user_code_ModelTrainer.egg-info
writing tfx_user_code_ModelTrainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_ModelTrainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_ModelTrainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_ModelTrainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_ModelTrainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_ModelTrainer.egg-info/SOURCES.txt'
Copying tfx_user_c

In [12]:
# TODO 3
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=PROJECT, location=REGION)

# Create a job to submit the pipeline
job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=f'{config.PIPELINE_NAME}',
                                parameter_values={
                                    'learning_rate': 0.003,
                                    'batch_size': 512,
                                    'num_epochs': 30,
                                })
job.submit()

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/13110252891/locations/us-central1/pipelineJobs/modifiedpipeline-20240126150513


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/13110252891/locations/us-central1/pipelineJobs/modifiedpipeline-20240126150513


To use this PipelineJob in another session:


INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:


pipeline_job = aiplatform.PipelineJob.get('projects/13110252891/locations/us-central1/pipelineJobs/modifiedpipeline-20240126150513')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/13110252891/locations/us-central1/pipelineJobs/modifiedpipeline-20240126150513')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/modifiedpipeline-20240126150513?project=13110252891


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/modifiedpipeline-20240126150513?project=13110252891
